In [71]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import statsmodels as sm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from statsmodels.tsa.arima_model import ARMA

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
with open('data_processed/data_feat.pickle', 'rb') as handle:
    data_feat = pickle.load(handle)
X_train = data_feat[0]
y_train = data_feat[1]
X_test = data_feat[2]
y_test = data_feat[3]

In [52]:
col_names = list(X_train.columns)

### Normalize Data

In [53]:
scaler_X = StandardScaler()
X_train_n = scaler_X.fit_transform(X_train)
X_test_n = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_n = scaler_y.fit_transform(np.array(y_train).reshape(-1,1))
y_test_n = scaler_y.transform(np.array(y_test).reshape(-1,1))

## Regression Models


In [54]:
### LASSO
lasso = LassoCV(cv=5, random_state=0, max_iter=10e4).fit(X_train_n, y_train_n)
train_rsq = lasso.score(X_train_n, y_train_n)
print('Train R-Squared: ', train_rsq)

y_hat = lasso.predict(X_test_n)
test_rsq = lasso.score(X_test_n, y_test_n)
print('Test R-Squared: ', test_rsq)

/Users/mschoder/.pyenv/versions/dev_374/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:1100: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train R-Squared:  0.5732725136911497
Test R-Squared:  0.5793971387553365


In [79]:
lasso.alpha_

0.00026706155288369127

In [65]:
y_train_n.ndim

2

## ARMAX 

In [73]:
model = ARMA(y_train_n, exog=X_train_n, order=(1, 1))
model_fit = model.fit(disp=True)

In [80]:
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                51318
Model:                     ARMA(1, 1)   Log Likelihood               13806.953
Method:                       css-mle   S.D. of innovations              0.185
Date:                Mon, 02 Dec 2019   AIC                         -27579.906
Time:                        00:10:16   BIC                         -27429.527
Sample:                             0   HQIC                        -27532.857
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.459e-05      0.022      0.002      0.998      -0.042       0.043
x1            -0.3310      0.047     -7.008      0.000      -0.424      -0.238
x2             0.0384      0.044      0.863      0.3

In [76]:
model_fit.predict(y_train, exog=X_train)

KeyError: 'The `start` argument could not be matched to a location related to the index of the data.'